In [1]:
import pandas as pd

### This code is for preprocessing the dataset and creating a smaller dataset.
My dataset has over 2 million records and it takes a lot of processing time so I have taken a sample of that data for this project.
I have chosen the first 70,000 records for my analysis as the system cannot process the original dataset

In [4]:
# Load the datasets.
questions = pd.read_csv("../data/Questions.csv", encoding="ISO-8859-1", nrows=70000)
answers = pd.read_csv("../data/Answers.csv", encoding="ISO-8859-1", nrows=70000)
tags = pd.read_csv("../data/Tags.csv", encoding="ISO-8859-1", nrows=70000)

Displaying the structure of the data to understand the relationships

In [7]:
print(questions.head())
print(answers.head())
print(tags.head())

answers.columns

    Id  OwnerUserId          CreationDate            ClosedDate  Score  \
0   80         26.0  2008-08-01T13:57:07Z                   NaN     26   
1   90         58.0  2008-08-01T14:41:24Z  2012-12-26T03:45:49Z    144   
2  120         83.0  2008-08-01T15:50:08Z                   NaN     21   
3  180    2089740.0  2008-08-01T18:42:19Z                   NaN     53   
4  260         91.0  2008-08-01T23:22:08Z                   NaN     49   

                                               Title  \
0  SQLStatement.execute() - multiple queries in o...   
1  Good branching and merging tutorials for Torto...   
2                                  ASP.NET Site Maps   
3                 Function for creating color wheels   
4  Adding scripting functionality to .NET applica...   

                                                Body  
0  <p>I've written a database generation script i...  
1  <p>Are there any really good tutorials explain...  
2  <p>Has anyone got experience creating <strong>... 

Index(['Id', 'OwnerUserId', 'CreationDate', 'ParentId', 'Score', 'Body'], dtype='object')

In [16]:
# Merge questions with tags
questions_with_tags = questions.merge(
    tags, left_on='Id', right_on='Id', how='inner'
)

# Merge questions with answers
questions_with_answers = questions_with_tags.merge(
    answers, left_on='Id', right_on='ParentId', how='inner',
    suffixes=('_question', '_answer')
)

# Sort answers by ParentId (question ID) and Score in descending order
sorted_answers = questions_with_answers.sort_values(
    by=['ParentId', 'Score_answer'], ascending=[True, False]
)

# Group by ParentId (question ID) and select only the top 2 answers
top_2_answers = sorted_answers.groupby('ParentId').head(2).reset_index(drop=True)

# Clean and preprocess the dataset
# Retain only the necessary columns and rename for clarity
final_dataset = top_2_answers[[
    'Id_question', 'Title', 'Body_question', 'Body_answer', 'Score_answer', 'Tag'
]].rename(columns={
    'Id_question': 'Question_ID',
    'Title': 'Question_Title',
    'Body_question': 'Question_Body',
    'Body_answer': 'Answer_Body',
    'Score_answer': 'Answer_Score'
})

# Remove duplicates
final_dataset = final_dataset.drop_duplicates(subset=['Question_ID', 'Answer_Body', 'Tag'])

# Standardize text formatting
final_dataset['Question_Body'] = final_dataset['Question_Body'].str.lower().str.strip()
final_dataset['Answer_Body'] = final_dataset['Answer_Body'].str.lower().str.strip()
final_dataset['Question_Title'] = final_dataset['Question_Title'].str.lower().str.strip()
final_dataset['Tag'] = final_dataset['Tag'].str.lower().str.strip()


In [18]:
# Save the dataset to a csv file
final_dataset.to_csv("../data/final_dataset.csv")

In [20]:
final_dataset.shape

(42244, 6)